In [12]:
import os#环境代理设置
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

- 其实是可以跑的，在autodl上成功了，不过是单核gpu，这里不知道哪出问题了
    - 不过8bit量化还是显存不够，改成4bit了，升维也取消了
- 别在本机运行TT
- 本机单核显存不够，但是明明都是24g显存，不知道为啥

In [13]:
import torch
import torch.nn as nn
import bitsandbytes as bnb 
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model 
model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-7b1", 
    load_in_4bit=True #load_in_8bit=True
)



The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.27s/it]


In [14]:
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-7b1")

In [15]:
# for i, param in enumerate(model.parameters()):
#     param.requires_grad = False
#     if param.ndim == 1: #把一维的小参数（bias、LayerNorm 等）单独升到 float32，保证数值稳定，其余权重保持原精度不动
#         param.data = param.data.to(torch.float32)

In [16]:
model.gradient_checkpointing_enable()  
model.enable_input_require_grads()


In [17]:
# class CastOutputToFloat(nn.Sequential):
#     def forward(self, x):
#         return super().forward(x).to(torch.float32) #将输出参数转为float32，权重参数不变

# model.lm_head = CastOutputToFloat(model.lm_head)

In [18]:
def print_trainable_parameters(mode):
    trainable_params = 0
    all_param = 0
    for _,param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"训练参数: {trainable_params} ||所有参数: {all_param} || 训练参数占比%: {100 * trainable_params / all_param}"
    )

In [19]:
from peft import LoraConfig, get_peft_model
config = LoraConfig(
    r=16, # 显存不够可以r=4
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)

print_trainable_parameters(model)

训练参数: 7864320 ||所有参数: 4056981504 || 训练参数占比%: 0.19384658254532677


In [9]:
import transformers
from datasets import load_dataset
dataset = load_dataset("Abirate/english_quotes")

def merge(row):
    row['prediction'] = row['quote'] + ' ->: ' + str(row['tags'])
    return row
dataset['train'] = dataset['train'].map(merge)

train_dataset = dataset.map(lambda x:tokenizer(x['prediction']), batched=True, remove_columns=['quote', 'author', 'tags', 'prediction'])

In [10]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
trainer = Trainer(
    model=model, 
    train_dataset=train_dataset['train'],
    args=TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="bloom-7b1-lora-0927"
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False 
trainer.train()


/root/miniconda3/envs/llama/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
1,3.405800
2,3.390400
3,2.763900
4,3.423100
5,2.927900
6,3.298700
7,3.352400
8,3.125000
9,3.122300
10,3.406800


TrainOutput(global_step=200, training_loss=2.319744032621384, metrics={'train_runtime': 391.7471, 'train_samples_per_second': 8.169, 'train_steps_per_second': 0.511, 'total_flos': 1.3292485294620672e+16, 'train_loss': 2.319744032621384, 'epoch': 1.2743221690590112})